# SemEval 2026 Task 8: Multi-Turn RAG Evaluation

## Complete Pipeline for All Tasks (Graph-Enhanced)

This pipeline generates submissions for Tasks A, B, and C.

| Task | Description | Method |
|------|-------------|--------|
| **A** | Retrieval | BGE-M3 + Cross-Encoder Reranking |
| **B** | Generation | Direct LLM (Llama 3.1) |
| **C** | RAG | Self-CRAG Graph with Hallucination Check |

---

## 1. Imports

In [1]:
import os, sys, json, zipfile
from tqdm import tqdm

# --- Project Root Detection ---
if os.path.exists("src"): PROJECT_ROOT = os.getcwd()
elif os.path.exists("llm-semeval-task8"): PROJECT_ROOT = "llm-semeval-task8"
else: PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path: sys.path.insert(0, PROJECT_ROOT)

# --- Ingestion & Retrieval ---
from src.ingestion import load_and_chunk_data, build_vector_store
from src.retrieval import get_retriever, get_qdrant_client

# --- Task B (Simple Gen) ---
from src.generation import create_generation_components
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# --- Task C (Advanced Graph) ---
from src.graph import initialize_graph

print(f"Project Root: {PROJECT_ROOT}")

/home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Project Root: /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8


---

## 2. Configuration

In [2]:
# ============================================================
# CONFIGURATION
# ============================================================
TEAM_NAME = "Gbgers"
DOMAINS = ["govt", "clapnq", "fiqa", "cloud"]
COLLECTION_NAME = "mtrag_unified"

# --- Execution Mode ---
# TEST_MODE = True  -> Fast validation (~5 min)
# TEST_MODE = False -> Full submission (~3-5 hours)
TEST_MODE = False

# --- Test Mode Limits ---
TEST_CHUNK_LIMIT = 1000      # Chunks per domain for indexing
TEST_QUERY_LIMIT = 5         # Conversations per domain

# --- Full Mode Limits ---
MAX_DOCS_PER_DOMAIN = 25000  # Max documents to load per domain

# --- Paths ---
CORPUS_DIR = os.path.join(PROJECT_ROOT, "dataset/corpora/passage_level")
CONV_FILE = os.path.join(PROJECT_ROOT, "dataset/human/conversations/conversations.json")
QDRANT_PATH = os.path.join(PROJECT_ROOT, "qdrant_db")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "data/submissions")

FILE_A = os.path.join(OUTPUT_DIR, f"submission_TaskA_{TEAM_NAME}.jsonl")
FILE_B = os.path.join(OUTPUT_DIR, f"submission_TaskB_{TEAM_NAME}.jsonl")
FILE_C = os.path.join(OUTPUT_DIR, f"submission_TaskC_{TEAM_NAME}.jsonl")

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(QDRANT_PATH, exist_ok=True)

print(f"Execution Mode: {'TEST' if TEST_MODE else 'FULL'}")
if not TEST_MODE:
    print(f"Max docs/domain: {MAX_DOCS_PER_DOMAIN}")

Execution Mode: FULL
Max docs/domain: 25000


---

## 3. Utility Functions

In [3]:
def extract_last_query(msgs):
    """Extract the most recent user query from a conversation."""
    return next((m["text"] for m in reversed(msgs) if m.get("speaker")=="user"), "")

def get_corpus(domain):
    """Get or extract corpus file path."""
    p = os.path.join(CORPUS_DIR, f"{domain}.jsonl")
    z = p + ".zip"
    if not os.path.exists(p) and os.path.exists(z):
        print(f"Extracting {domain}.jsonl...")
        with zipfile.ZipFile(z) as zf: zf.extractall(CORPUS_DIR)
    return p if os.path.exists(p) else None

def save_jsonl(data, path):
    """Save list of dicts to JSONL file."""
    with open(path, 'w', encoding='utf-8') as f:
        for d in data: f.write(json.dumps(d, ensure_ascii=False)+'\n')
    print(f"Saved {len(data)} items -> {path}")

---

## 4. Build Unified Vector Index

In [4]:
# --- Check if Index Exists ---
need_build = True
try:
    client = get_qdrant_client(QDRANT_PATH)
    if client.collection_exists(COLLECTION_NAME):
        info = client.get_collection(COLLECTION_NAME)
        print(f"Collection '{COLLECTION_NAME}' found: {info.points_count} vectors")
        need_build = False
except Exception as e:
    print(f"Note: {e}")

# --- Build if Needed ---
if need_build:
    print(f"Building '{COLLECTION_NAME}'...")
    all_docs = []
    limit = TEST_CHUNK_LIMIT if TEST_MODE else MAX_DOCS_PER_DOMAIN
    
    for domain in DOMAINS:
        path = get_corpus(domain)
        if not path:
            print(f"Warning: Corpus not found for {domain}")
            continue
        print(f"Loading {domain}...")
        docs = load_and_chunk_data(path)
        for d in docs: d.metadata["domain"] = domain
        
        if len(docs) > limit:
            print(f"  Limiting: {len(docs)} -> {limit}")
            docs = docs[:limit]
        
        all_docs.extend(docs)
        print(f"  Added {len(docs)} chunks")
    
    print(f"Total: {len(all_docs)} chunks")
    build_vector_store(all_docs, persist_dir=QDRANT_PATH, collection_name=COLLECTION_NAME)
    print("Index built successfully.")

Collection 'mtrag_unified' found: 100000 vectors


/home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/src/retrieval.py:28: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Collection <mtrag_unified> contains 100000 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  _qdrant_client = QdrantClient(path=qdrant_path)


---

## 5. Initialize Components (Retriever, LLM, Graph)

In [5]:
# --- 5.1 Initialize Retriever ---
print("Loading Retriever...")
retriever = get_retriever(qdrant_path=QDRANT_PATH, collection_name=COLLECTION_NAME)

# --- 5.2 Initialize LLM for Task B ---
print("Loading LLM for Task B...")
gen_components = create_generation_components()

# --- 5.3 Initialize Self-CRAG Graph for Task C ---
print("Initializing Advanced Graph for Task C...")
# CRITICAL FIX: Share the SAME generation components with the graph to avoid OOM
# Otherwise, the graph would load a SECOND copy of the model.
import src.graph
src.graph._components = gen_components
graph_app = initialize_graph()

# --- 5.4 Create Task B Chain (no context) ---
task_b_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert assistant. Answer based on your knowledge. Be concise.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"]
)
task_b_chain = task_b_prompt | gen_components.llm | StrOutputParser()

print("All Systems Ready.")

Loading Retriever...
Loading reranker: BAAI/bge-reranker-v2-m3
Loading LLM for Task B...
Creating Generation Components with model: meta-llama/Llama-3.1-8B-Instruct...


Loading checkpoint shards: 100%|██████████| 4/4 [00:22<00:00,  5.52s/it]
Device set to use cuda:0


Generation Components Ready.
Initializing Advanced Graph for Task C...
All Systems Ready.


---

## 6. Execute Unified Pipeline

This cell processes all conversations and generates results for all three tasks.

**Note on `conversation_id`:** The dataset does NOT have a native `conversation_id` key. 
We generate a unique ID using `{domain}_{index}` format (e.g., `govt_0`, `govt_1`, ...).

In [6]:
# --- Load Conversations ---
print("Loading conversations...")
with open(CONV_FILE) as f:
    conversations = json.load(f)
print(f"Total conversations: {len(conversations)}")

# --- Analyze JSON structure (first run info) ---
print(f"\nJSON Keys: {list(conversations[0].keys())}")

# --- Initialize Result Containers ---
results_A, results_B, results_C = [], [], []

# --- Process Each Domain ---
for domain in DOMAINS:
    print(f"\n{'='*50}\n DOMAIN: {domain.upper()}\n{'='*50}")
    
    # Filter by domain (substring match on the 'domain' key), keeping track of global index
    convs_with_idx = []
    for i, c in enumerate(conversations):
        if domain in c.get("domain", "").lower():
            convs_with_idx.append((i, c))
            
    if TEST_MODE:
        convs_with_idx = convs_with_idx[:TEST_QUERY_LIMIT]
    print(f"Processing {len(convs_with_idx)} conversations")
    
    for idx_in_domain, (global_idx, conv) in enumerate(tqdm(convs_with_idx, desc=domain)):
        msgs = conv.get("messages", [])
        q = extract_last_query(msgs)
        if not q:
            continue
        
        # --- Generate unique conversation_id ---
        # Format: {domain}_{index_within_domain}
        conv_id = f"{domain}_{idx_in_domain}"
        
        # -------------------------------------------------------------------
        # TASK A: Retrieval
        # -------------------------------------------------------------------
        docs = retriever.invoke(q)
        contexts = []
        for i, d in enumerate(docs):
            txt = d.metadata.get("parent_text") or d.page_content
            contexts.append({
                "document_id": str(d.metadata.get("doc_id", f"{domain}_{i}")),
                "score": float(d.metadata.get("relevance_score", 0.0)),
                "text": txt
            })
        
        # -------------------------------------------------------------------
        # TASK B: Generation (Direct LLM, No Context)
        # -------------------------------------------------------------------
        try:
            ans_b = task_b_chain.invoke({"question": q})
        except Exception as e:
            ans_b = str(e)
        
        # -------------------------------------------------------------------
        # TASK C: RAG Generation (Self-CRAG Graph)
        # -------------------------------------------------------------------
        try:
            # Convert raw msgs to LangChain objects for usage in Rewrite Node
            chat_history = []
            for m in msgs:
                role = m.get("speaker", "user")
                content = m.get("text", "")
                if role == "user":
                    chat_history.append(HumanMessage(content=content))
                else:
                    chat_history.append(AIMessage(content=content))
            
            # Pass messages to graph for rewriter
            response = graph_app.invoke({
                "question": q, 
                "domain": domain,
                "messages": chat_history
            })
            ans_c = response.get("generation", "I_DONT_KNOW")
            reason_c = response.get("fallback_reason", "none")
            docs_relevant_c = response.get("documents_relevant", "unknown")
            is_hallucination_c = response.get("is_hallucination", "unknown")
            retry_count_c = response.get("retry_count", 0)
        except Exception as e:
            print(f"Task C Error: {e}")
            ans_c = "I_DONT_KNOW"
            
        # -------------------------------------------------------------------
        # Append Results
        # -------------------------------------------------------------------
        results_A.append({"conversation_id": conv_id, "original_index": global_idx, "ranking": contexts})
        results_B.append({"conversation_id": conv_id, "original_index": global_idx, "answer": ans_b})
        results_C.append({"conversation_id": conv_id, "original_index": global_idx, "answer": ans_c, "fallback_reason": reason_c, "docs_relevant": docs_relevant_c, "is_hallucination": is_hallucination_c, "retry_count": retry_count_c, "references": contexts})
        
print(f"\nProcessing Complete. Total Results: {len(results_A)}")

Loading conversations...
Total conversations: 110

JSON Keys: ['author', 'retriever', 'generator', 'messages', 'status', 'status_history', 'editor', 'domain', 'reviewer']

 DOMAIN: GOVT
Processing 28 conversations


govt:   0%|          | 0/28 [00:00<?, ?it/s]

Initializing retriever for domain: govt


govt: 100%|██████████| 28/28 [08:55<00:00, 19.11s/it]



 DOMAIN: CLAPNQ
Processing 29 conversations


...


 DOMAIN: CLOUD
Processing 26 conversations


cloud:   0%|          | 0/26 [00:00<?, ?it/s]

Initializing retriever for domain: cloud


cloud: 100%|██████████| 26/26 [08:48<00:00, 20.34s/it]


Processing Complete. Total Results: 110


In [8]:
print("Saving submission files...")

save_jsonl(results_A, FILE_A)
save_jsonl(results_B, FILE_B)
save_jsonl(results_C, FILE_C)

print("\nAll Done!")

Saving submission files...
Saved 110 items -> /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/data/submissions/submission_TaskA_Gbgers.jsonl
Saved 110 items -> /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/data/submissions/submission_TaskB_Gbgers.jsonl
Saved 110 items -> /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8/data/submissions/submission_TaskC_Gbgers.jsonl

All Done!
